<h1>ECON 140R Class 11.2</h1>

This is an update of the Class 10.2 hands-on:
* 11.1: Working with IPUMS data, like an extract from the January 2018 CPS
* <b>11.2: Bad controls in the January 2018 Current Population Survey (CPS)</b>

where I have fixed a recoding error in `edyrs` that I discovered while writing Class 11.1.

Learning objectives:

1. More experience with OLS
2. Handy `factor()` function
3. Bad controls may cut off the avenues through which the treatment affects the outcome
4. Balancing omitted variable bias vs. bad controls is difficult

<h2>Bad controls</h2>

Let's examine how controlling for occupation and industry, while feasible to do, changes the estimated coefficients on education and female gender identity. It's probably not a good idea, unless there were a compelling reason to look at within-occupation differences in earnings by education. 

To explore this topic, we'll look at an extract from the January 2018 Current Population Survey (CPS), the annual job tenure supplement. I downloaded the extract from [IPUMS](http://cps.ipums.org), and in the future we'll walk through the IPUMS interface and contents.

In [ ]:
library(tidyverse)
library(haven)

It turns out that this handy command stops __R__ from defaulting to scientific notation. 

In [ ]:
options(scipen=999)

This extract contains people aged 25-64, which are working ages when education is likely to be completed.

In [ ]:
# This is the updated file with corrected edyrs. There had been a miscoding for EDUC == 81
cpsj18_2564 = read_dta("cpsj18_2564_1.dta")
head(cpsj18_2564)

Here is a baseline regression, where we are controlling for 0/1 female gender identity and years of education, and for a set of controls including race/ethnicity, years of experience, years of experience squared, and also years of job tenure:

$$
\ln Y_i = \alpha + \beta^f \ female_i + \beta^e \ educ_i + B \ controls_i + \epsilon_i
$$

We'll run this regression and examine what we find for $\beta^f$ and $\beta^e$:

In [ ]:
cps_reg1 <- lm(logearnweek ~ female + edyrs + factor(raceth) + exper + expersq + jtyears, 
               data = cpsj18_2564)
summary(cps_reg1)

We see a pretty large negative penalty for female workers: $-$37%, and we see a relatively large benefit for an additional year of education: 11%. Racial/ethnic minorities see earnings penalties relative to white non-Hispanic people (the baseline group). Log earnings are a parabola opening down in years of experience, which is consistent with typical theory. Years of job tenure is a benefit, with each year giving us 1.7% more earnings 

Now let's explore controlling for occupation `occ`, and for kicks, also for industry `ind` and state of residence `statefip`. The last one is probably not a bad control, come to think of it, unless one believes that education's effect on earning is partially through enhanching geographic mobility, which isn't an impossibility. 

Let's try including just `statefip` first: 

In [ ]:
cps_reg2 <- lm(logearnweek ~ female + edyrs + factor(raceth) + exper + expersq + jtyears
               + factor(statefip),
               data = cpsj18_2564)
summary(cps_reg2)

We do not see much of a difference here compared to the first regression. The coefficient on female, $\beta^f$, shows a penalty of about $-$36%; and the coefficient on years of education, $\beta^e$, shows a benefit of additional year is 10.2%.

But what about the coefficients on the racial and ethnic identity indicators? Do you see anything different there? What do you think might be going on?

<hr>

Now let's throw the kitchen sink at things.

In these data, there are almost 500 occupation categories, and there are roughly 250 industries. For details, see the documentation at IPUMS: [OCC documentation](https://cps.ipums.org/cps/codes/occ_20112019_codes.shtml), [IND documentation](https://cps.ipums.org/cps/codes/ind_2014_codes.shtml).

In [ ]:
cps_reg3 <- lm(logearnweek ~ female + edyrs + factor(raceth) + exper + expersq + jtyears
               + factor(occ) + factor(ind) + factor(statefip)
               , data = cpsj18_2564)
summary(cps_reg3)

What do we see here compared to earlier results?

The coefficient on female identity is now about $\beta^f = -0.24$, smaller in magnitude by a factor of about a third. In other words, within occupation-industry-state groupings in the data, female workers earn on average 24% less than observationally identical males.

Meanwhile, the coefficient on years of education has fallen by roughly half, down to $\beta^e = 0.05$. Here too, the story is that within occupation-industry-state groupings of workers, the returns to an additional year of education are smaller than they are overall.

<div style="text-align: right"> <span style="font-family:Papyrus; ">And they lived happily ever after. The End.</span></div>